In [ ]:

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from mpl_point_clicker import clicker
from matplotlib.path import Path

# Configuración para la visualización
plt.style.use('seaborn-darkgrid')


In [ ]:

# Cargar los datos
arm1_data = pd.read_csv('../DATA/processed/TracingPoints/arm_tracing_11_arm1_rho_dani.csv')
arm2_data = pd.read_csv('../DATA/processed/TracingPoints/arm_tracing_11_arm2_rho_dani.csv')
data = pd.read_csv('../DATA/processed/TracingPoints/data_rho_11_filtered.csv')

# Calcular coordenadas polares (r, theta)
for df in [data, arm1_data, arm2_data]:
    df['r'] = np.sqrt(df['x']**2 + df['y']**2)
    df['theta'] = np.degrees(np.arctan2(df['y'], df['x']))


In [ ]:

plt.figure(figsize=(10, 6))
plt.scatter(data['theta'], data['r'], alpha=0.5, s=0.5, label='Datos')
plt.scatter(arm1_data['theta'], arm1_data['r'], alpha=0.7, s=10, label='Brazo 1')
plt.scatter(arm2_data['theta'], arm2_data['r'], alpha=0.7, s=10, label='Brazo 2')

plt.title("Datos iniciales")
plt.xlabel("Theta (grados)")
plt.ylabel("r")
plt.legend()
plt.show()


In [ ]:

def seleccionar_ajustar(data_combined, id_arm):
    print(f"Seleccionando puntos para el brazo {id_arm}")

    fig, ax = plt.subplots(figsize=(10, 6))
    scatter = ax.scatter(data_combined['theta'], data_combined['r'], s=0.5, alpha=0.5)
    ax.set_title(f"Seleccionar puntos para el brazo {id_arm}")
    ax.set_xlabel("Theta (grados)")
    ax.set_ylabel("r")
    klicker = clicker(ax, ["limits"], markers=["o"], colors=['red'], markersize=5)
    plt.show()

    dict_lims = klicker.get_positions()
    if "limits" not in dict_lims or len(dict_lims["limits"]) < 3:
        print("Selecciona al menos tres puntos.")
        return None, None

    polygon_path = Path(dict_lims["limits"])
    inside_points = data_combined[data_combined.apply(lambda row: polygon_path.contains_point((row['theta'], row['r'])), axis=1)]
    
    if len(inside_points) > 1:
        model = LinearRegression()
        model.fit(inside_points[['theta']], inside_points['r'])
        slope, intercept = model.coef_[0], model.intercept_
        return inside_points, (slope, intercept)
    else:
        print("Selecciona más puntos para realizar el ajuste.")
        return None, None


In [ ]:

data_combined = pd.concat([data[['theta', 'r']], arm1_data[['theta', 'r']], arm2_data[['theta', 'r']]], ignore_index=True)
brazo1_points, brazo1_fit = seleccionar_ajustar(data_combined, id_arm=1)
brazo2_points, brazo2_fit = seleccionar_ajustar(data_combined, id_arm=2)


In [ ]:

fig, ax = plt.subplots(figsize=(10, 6))
ax.scatter(data['theta'], data['r'], s=0.5, alpha=0.5, label='Datos originales')
colors = ['purple', 'green']

for i, (points, fit) in enumerate([(brazo1_points, brazo1_fit), (brazo2_points, brazo2_fit)]):
    if points is not None:
        ax.scatter(points['theta'], points['r'], s=10, alpha=0.7, label=f'Brazo {i + 1}', color=colors[i])
        theta_range = np.linspace(points['theta'].min(), points['theta'].max(), 100)
        r_fit = fit[0] * theta_range + fit[1]
        ax.plot(theta_range, r_fit, label=f'Ajuste Brazo {i + 1}', color=colors[i])

ax.set_title("Ajustes de los brazos")
ax.set_xlabel("Theta (grados)")
ax.set_ylabel("r")
ax.legend()
plt.show()
